# Ollama를 활용한 도구 호출 에이전트(Tool Calling Agent)

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
secret_env = os.getenv("OPENAI_BASE_URL")
print(secret_env)

https://127.0.0.1:11434/v1/


In [6]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

# 도구 정의 
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword


In [7]:
# tools 정의 
tools = [search_news]

## Agent용 프롬프트 생성
- `chat_history`: 이전 대화 내용을 저장하는 변수(멀티턴을 지원하지 않으면 사용하지 않아도 됨)
- `agent_scratchpad`: 에이전트가 임시로 저장하는 변수
- `input`: 사용자의 입력

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공(도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant."
            "Make sure to use the 'search_news' tool for searching keyword related news."
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

## Tool Calling을 지원하는 Ollama 목록

- [Ollama Tool Calling 지원 모델 리스트](https://ollama.com/search?c=tools)
- Ollama가 설치되어져 있지 않다면 [Ollama](https://wikidocs.net/233805) 에서 설치
- 터미널 창에 `ollama pull llama3.1` 명령어를 입력하여 모델을 다운

In [9]:
from langchain_ollama import ChatOllama
import os

# Llama-3.1
ollama = ChatOllama(model="llama3.1", temperature=0)

# Qwen2.5 7B
qwen = ChatOllama(
    model="qwen2.5:latest"
)

llm_chat = ChatOllama(model="llama2:7b-chat")

In [10]:
print(llm_chat)

model='llama2:7b-chat'


In [11]:
from langchain.agents import create_tool_calling_agent

ollama_agent = create_tool_calling_agent(ollama, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)
llamaChat_agent = create_tool_calling_agent(llm_chat, tools, prompt)

## AgentExecutor 생성 후 실행 및 결과 확인

In [12]:
from langchain.agents import AgentExecutor

# gpt 사용하는 코드는 넣지 않아서 해당 코드는 실행되지 않음
# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=qwen_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE1CQU5NRmVvZ0VjNzl3amRVbXlXVU1VOHZNbjIwVEJITmMzNlBYTEdybmxRTWlPWktJODJSZTNWNHdZSGdFY3V0Qnh0OUhkUjl6VnBZVTdR0gFXQVVfeXFMTm5WNXJ6OGcwLWYxelU1SzVGdkt5Q1JJdjNmQlBQZE9aX3hiQjZVR2JnQ2hkOEd2ZEUyTHVFSnE1eGs1RUtyb2dqeDBHZzNwR29vSjVXYXQ4?oc=5', 'content': "AI 인프라 솔루션 '텐', 누적 투자금 112억 달성 [투자ABC] - 서울경제"}, {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE0ya3RMM1JhTGFnLXJZZ19oRS1qQzR5OWJNUUtoeW1rYVh6ZWRIWUMxWnVNYnR5RVo2MVlJWlhJNnFHTWtMVVBRQVg3OUl3RFV5cUxpWE91YjdyZ9IBVEFVX3lxTE9Rc2dnYkgyV3JOdFV3MmhoZGZESVVMSUFPX3I1OW9HeXZEbURGT0lFbFNrS25EQ2JYV3RZTjFzWmtraUJYcWpnS1drVWloaS0xazY3QQ?oc=5', 'content': '"구글 검색사업, AI로 돌파구 찾을 것"...투자고수, 알파벳 집중매수 - 한국경제'}, {'url': 'https://news.google.com/rss/articles/CBMihwFBVV95cUxNR2dOV28zdVk5Q1lLS2wtXzdKTm1WSTVwRS1KMWtnN212b1lObHVBSno5R3EwN2Q1bDZ6c09STVRHaXNNekRkZVNYeTJiRjNyQ1U5bS1tX0lkM25STmhnUlNFSGFZcUxsS1

아래는 입력받은 llm을 사용하여 Agent를 생성하고 실행하여 결과를 출력하는 함수

In [119]:
from langchain.agents import AgentExecutor

def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

In [120]:
from langchain.agents import initialize_agent, AgentType

agent_executor = initialize_agent(
    tools,
    llm_chat,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent_executor.run("Hello!")



> Entering new AgentExecutor chain...


AttributeError: 'FieldInfo' object has no attribute 'chat'

In [121]:
query = "AI 투자와 관련된 뉴스를 검색하고 결과를 Instagram 게시글 형식으로 작성해 주세요. 한글로 답변하세요!"

In [126]:
execute_agent(qwen, tools, query, "qwen2.5(Ollama)")

AttributeError: 'FieldInfo' object has no attribute 'chat'

In [127]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.1(Ollama)")

AttributeError: 'FieldInfo' object has no attribute 'chat'

In [128]:
execute_agent(llm_chat, tools, query, "llama2:7b-chat(Ollama)")

AttributeError: 'FieldInfo' object has no attribute 'chat'

# TEST

In [130]:
from langchain_ollama import ChatOllama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# ChatOllama 모델 초기화
llm = ChatOllama(model="qwen2.5")  # 올바른 모델 이름 사용

# 예시 도구 정의
tools = [
    Tool(
        name="MyTool",
        func=lambda x: f"Processed: {x}",
        description="Example tool for processing input"
    )
]

# 에이전트 실행기 초기화
agent_executor = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 에이전트 실행
query = "AI 투자와 관련된 뉴스를 검색하고 결과를 Instagram 게시글 형식으로 작성해 주세요. 한글로 답변하세요!"
result = agent_executor.run(query)
print(f"[qwen2.5(Ollama)] 결과입니다: {result}")




> Entering new AgentExecutor chain...


AttributeError: 'FieldInfo' object has no attribute 'chat'

In [132]:
import ollama

response = ollama.chat(
    model='llama3.1:latest',
    messages=[{'role': 'user', 'content':
        'What is the weather in Toronto?'}],

	# provide a weather checking tool to the model
    tools=[{
      'type': 'function',
      'function': {
        'name': 'get_current_weather',
        'description': 'Get the current weather for a city',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'The name of the city',
            },
          },
          'required': ['city'],
        },
      },
    },
  ],
)

print(response['message']['tool_calls'])

[ToolCall(function=Function(name='get_current_weather', arguments={'city': 'Toronto'}))]


In [1]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    base_url="http://localhost:11434",
    model="llama3.1:latest",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

[{'name': 'validate_user',
  'args': {'addresses': ['123 Fake St, Boston, MA',
    '234 Pretend Boulevard, Houston, TX'],
   'user_id': 123},
  'id': '2a1088a5-7f81-4ea4-9dae-916ffd709a79',
  'type': 'tool_call'}]

In [149]:
from langchain.tools import Tool

def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True
    
tool = Tool(
    name="validate_user",
    func=validate_user,
    description="Validate user using historical addresses."
)

llm = ChatOllama(
    model="llama3:latest",
    temperature=0
).bind_tools([tool])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
print(result)

AttributeError: 'FieldInfo' object has no attribute 'chat'